In [ ]:
import torch
import torch.linalg as tla
import numpy as np
import matplotlib.pyplot as plt
import numml.sparse as sp

In [ ]:
# Create our favorite poisson operator

N = 16
A = sp.eye(N)*2. - sp.eye(N, k=1) - sp.eye(N, k=-1)
print(A.to_dense())

In [ ]:
# A row-wise weighted jacobi, where instead of a scalar omega we have an n-length vector
# weighting each entry

def node_weighted_jacobi(A, x, b, omega):
    Dinv = sp.diag(1./A.diagonal())
    return omega * (Dinv @ b) + (sp.eye(A.shape[0]) - sp.diag(omega) @ (Dinv@A)) @ x

In [ ]:
# We'll optimize over the best weight for each entry

b = torch.zeros(N)
omega = torch.ones(N, requires_grad=True)

In [ ]:
optimizer = torch.optim.Adam([omega], lr=1e-2)
batch = 20
lh = []
oh = []
jac_iter = 1

for i in range(100):
    optimizer.zero_grad()
    
    loss = 0.
    
    for j in range(batch):
        x = torch.rand(N)
        x /= tla.norm(x)
        for k in range(jac_iter):
            x = node_weighted_jacobi(A, torch.rand(N), b, omega)
        loss += torch.sum((b - A @ x)**2.) / batch
    
    loss.backward()
    
    optimizer.step()
    
    lh.append(loss.item())
    oh.append(omega.detach().numpy().copy())
    if i % 10 == 0:
        print(i, 'loss:', round(loss.item(), 3), 'omega:', torch.round(omega.detach(), decimals=2))

In [ ]:
Lh = np.array(lh)
Oh = np.array(oh)

plt.figure(figsize=(6, 5))

plt.figure()
plt.plot(Lh, 'k')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.grid()

ax2 = plt.gca().twinx()
ax2.plot(Oh)
ax2.set_ylabel('Jacobi Weight')

In [ ]:
plt.figure(figsize=(6, 3))
plt.title('Entry-wise jacobi weights')
plt.plot(omega.detach(), 'o-')
plt.ylim(0.3, 1)
plt.xlabel('Node')
plt.ylabel('Jacobi Weight')
plt.grid()